# 导包

In [34]:
import datetime 
import pandas as pd
import numpy as np
from math import log,sqrt,exp
from scipy import stats
import plotly.graph_objects as go
import plotly
import plotly.express as px
import re
plotly.offline.init_notebook_mode(connected=True)

# 50ETF期权日行情2015-2018日行情数据

分别读取2015-2018年的50ETF期权合约数据

In [35]:
df = pd.read_csv('./data/50ETF期权日行情2017-2018.csv',usecols=['date','trade_code','收盘价','成交量'])
df2016 = pd.read_csv('./data/50ETF期权日行情2015-2016.csv',usecols=['date','trade_code','收盘价','成交量'],encoding='gbk')
df = df.append(df2016)
# 细节更正
df.date = pd.to_datetime(df.date) # 把date列改为datetime类型，否则sort排序出错
df = df.sort_values('date').reset_index(drop=True) # 按日期排序

# 用于展平DataFrame
pd.set_option('display.max_columns', None)

In [36]:
df

,date,trade_code,收盘价,成交量
0,2015-02-09,510050C1509M02200,0.3130,247
1,2015-02-09,510050C1506M02250,0.2366,114
2,2015-02-09,510050C1506M02300,0.2105,161
3,2015-02-09,510050C1506M02350,0.1870,106
4,2015-02-09,510050P1506M02200,0.1379,105
...,...,...,...,...
98749,2018-09-21,510050P1809M02450,0.0005,"46,598"
98750,2018-09-21,510050C1809M02450,0.1749,"48,142"
98751,2018-09-21,510050P1809M02500,0.0008,"105,318"
98752,2018-09-21,510050P1812M02550,0.0709,"3,262"


In [37]:
df['K'] = [int(i[-4:])/1000 for i in df.trade_code] # 获取行权价格
df['call_put'] = [i[6] for i in df.trade_code]  # 分离看涨、看跌
df['Adjust'] = [i[11] for i in df.trade_code]  # 分离看涨、看跌
# 弃用
# df.date = pd.to_datetime(df.date) # 转变日期格式 
# pd.to_numeric(df['成交量'])
df['成交量'] = df['成交量'].astype('str').apply(lambda x: x.replace(',','')).astype('int') # 成交量含有非法字符“，”，去除后转换为整型

# 成交量剔除

成交量过小说明流动性差，直接剔除（以20为界）

In [38]:
df[df['成交量'] < 20]

,date,trade_code,收盘价,成交量,K,call_put,Adjust
248,2015-02-16,510050C1509M02200,0.3120,2,2.20,C,M
253,2015-02-16,510050P1509M02250,0.0998,5,2.25,P,M
256,2015-02-16,510050C1509M02300,NaN,0,2.30,C,M
257,2015-02-16,510050C1509M02250,NaN,0,2.25,C,M
271,2015-02-16,510050P1506M02500,0.1837,16,2.50,P,M
...,...,...,...,...,...,...,...
94673,2018-08-08,510050P1809M03500,0.9950,4,3.50,P,M
94674,2018-08-08,510050P1809M03400,0.8702,6,3.40,P,M
94675,2018-08-08,510050P1809M03300,0.7850,5,3.30,P,M
94676,2018-08-08,510050P1809M03200,0.6933,4,3.20,P,M


In [39]:
df = df[df['成交量'] >= 20] # df['成交量'].quantile(0.1)

In [40]:
df

,date,trade_code,收盘价,成交量,K,call_put,Adjust
0,2015-02-09,510050C1509M02200,0.3130,247,2.20,C,M
1,2015-02-09,510050C1506M02250,0.2366,114,2.25,C,M
2,2015-02-09,510050C1506M02300,0.2105,161,2.30,C,M
3,2015-02-09,510050C1506M02350,0.1870,106,2.35,C,M
4,2015-02-09,510050P1506M02200,0.1379,105,2.20,P,M
...,...,...,...,...,...,...,...
98749,2018-09-21,510050P1809M02450,0.0005,46598,2.45,P,M
98750,2018-09-21,510050C1809M02450,0.1749,48142,2.45,C,M
98751,2018-09-21,510050P1809M02500,0.0008,105318,2.50,P,M
98752,2018-09-21,510050P1812M02550,0.0709,3262,2.55,P,M


# 收盘价剔除

In [41]:
df[df['收盘价'] < 0.0001]

,date,trade_code,收盘价,成交量,K,call_put,Adjust


In [42]:
df = df[df['收盘价'] >= 0.0001]
df

,date,trade_code,收盘价,成交量,K,call_put,Adjust
0,2015-02-09,510050C1509M02200,0.3130,247,2.20,C,M
1,2015-02-09,510050C1506M02250,0.2366,114,2.25,C,M
2,2015-02-09,510050C1506M02300,0.2105,161,2.30,C,M
3,2015-02-09,510050C1506M02350,0.1870,106,2.35,C,M
4,2015-02-09,510050P1506M02200,0.1379,105,2.20,P,M
...,...,...,...,...,...,...,...
98749,2018-09-21,510050P1809M02450,0.0005,46598,2.45,P,M
98750,2018-09-21,510050C1809M02450,0.1749,48142,2.45,C,M
98751,2018-09-21,510050P1809M02500,0.0008,105318,2.50,P,M
98752,2018-09-21,510050P1812M02550,0.0709,3262,2.55,P,M


# 标的资产价格 50ETF

In [43]:
df_d = pd.read_csv('./data/50ETF基金净值表现日数据.csv',usecols=['date','close'],encoding='gbk')

In [44]:
df_d[df_d[df_d.date=='2015-02-09'].index.tolist()[0]: (df_d[df_d.date=='2018-09-21'].index.tolist()[0]+1)]
df_d.date = pd.to_datetime(df_d.date) # 转变日期格式

In [45]:
df_d

,date,close
0,2013-09-30,1.697
1,2013-10-08,1.717
2,2013-10-09,1.723
3,2013-10-10,1.694
4,2013-10-11,1.725
...,...,...
1221,2018-09-20,2.541
1222,2018-09-21,2.632
1223,2018-09-25,2.600
1224,2018-09-26,2.638


In [46]:
df = pd.merge(df,df_d,on='date',how='left') # 加上close列

In [47]:
df

,date,trade_code,收盘价,成交量,K,call_put,Adjust,close
0,2015-02-09,510050C1509M02200,0.3130,247,2.20,C,M,2.331
1,2015-02-09,510050C1506M02250,0.2366,114,2.25,C,M,2.331
2,2015-02-09,510050C1506M02300,0.2105,161,2.30,C,M,2.331
3,2015-02-09,510050C1506M02350,0.1870,106,2.35,C,M,2.331
4,2015-02-09,510050P1506M02200,0.1379,105,2.20,P,M,2.331
...,...,...,...,...,...,...,...,...
93097,2018-09-21,510050P1809M02450,0.0005,46598,2.45,P,M,2.632
93098,2018-09-21,510050C1809M02450,0.1749,48142,2.45,C,M,2.632
93099,2018-09-21,510050P1809M02500,0.0008,105318,2.50,P,M,2.632
93100,2018-09-21,510050P1812M02550,0.0709,3262,2.55,P,M,2.632


# 无风险利率替代品 shibor一周利率

In [52]:
df2 = pd.read_csv('./data/shibor.csv',usecols=['date','1w']) # 2017年
df3 = pd.read_csv('./data/shibor2.csv',usecols=['date','1w']) # 2018年
df4 = pd.read_csv('./data/shibor3.csv',usecols=['date','1w']) # 2015年
df5 = pd.read_csv('./data/shibor4.csv',usecols=['date','1w']) # 2016年
df_shibor = df2.append([df3,df4,df5]) # 拼接4年年
df_shibor = df_shibor.sort_values(by='date',ascending=True).reset_index(drop=True) # 重新排序
df_shibor.date = df_shibor.date.apply(lambda x: str(x)[:4]+'/'+str(x)[4:6]+'/'+str(x)[6:])# 日期格式
df_shibor.date = pd.to_datetime(df_shibor.date) # 转换时间格式

In [53]:
df_shibor

,date,1w
0,2015-01-04,4.883
1,2015-01-05,4.725
2,2015-01-06,4.241
3,2015-01-07,3.873
4,2015-01-08,3.745
...,...,...
992,2018-12-24,2.659
993,2018-12-25,2.653
994,2018-12-26,2.672
995,2018-12-27,2.695


In [54]:
df['shibor'] = df.date.apply(lambda x: (df_shibor[df_shibor.date==x]['1w'].values[0])/100) # 进行配对，将shibor利率放入df
# 弃用
# df['shibor'] = df['shibor'].apply(lambda x: x/100) # 利率以百分比形式展现
# df = df.sort_values(by='date').reset_index(drop=True) # 升序排列

In [55]:
df

,date,trade_code,收盘价,成交量,K,call_put,Adjust,close,shibor
0,2015-02-09,510050C1509M02200,0.3130,247,2.20,C,M,2.331,0.04335
1,2015-02-09,510050P1504M02400,0.1903,395,2.40,P,M,2.331,0.04335
2,2015-02-09,510050C1506M02250,0.2366,114,2.25,C,M,2.331,0.04335
3,2015-02-09,510050C1506M02300,0.2105,161,2.30,C,M,2.331,0.04335
4,2015-02-09,510050C1506M02350,0.1870,106,2.35,C,M,2.331,0.04335
...,...,...,...,...,...,...,...,...,...
93097,2018-09-21,510050C1812M02500,0.1975,3615,2.50,C,M,2.632,0.02654
93098,2018-09-21,510050P1809M02450,0.0005,46598,2.45,P,M,2.632,0.02654
93099,2018-09-21,510050C1809M02450,0.1749,48142,2.45,C,M,2.632,0.02654
93100,2018-09-21,510050C1809M02550,0.0783,243099,2.55,C,M,2.632,0.02654


## 到期日

In [56]:
df['t'] = [i[-10:-6] for i in df.trade_code]
fd = df.groupby('t').count().index.to_list()

In [57]:
def prompt_day(fd):
    td = []
    for d in fd:
        y = int('20' + d[:2]) 
        m = int(d[2:])
        if m != 12:
            ndays = (datetime.date(y, m + 1, 1) - datetime.date(y, m, 1)).days
        else: ndays = 31
        date_list = []
        for i in range(1,ndays+1):
            p = datetime.date(y,m,i)
            nweek = p.isoweekday() #返回数字1-7，1为周一,7为周日
            if nweek == 3:    # 交割日为每月第四周星期三
                d2 = p.strftime('%Y%m%d')
                date_list.append(d2)
            if i == ndays:
                td.append(date_list[3])
                date_list=[]  # 清空
    return td
td = prompt_day(fd)

# 剔除交割日期少于7天

In [58]:
df2 = df.copy()
df2.t = pd.to_datetime([i for x in df2.t for i in td if re.search(x,i) != None])
df2.t = df2.t - df2.date
df2[df2.t.astype('str').apply(lambda x:x[:-5]).astype('int32') < 7]

,date,trade_code,收盘价,成交量,K,call_put,Adjust,close,shibor,t
1238,2015-03-19,510050P1503M02500,0.0063,503,2.50,P,M,2.587,0.04496,6 days
1244,2015-03-19,510050C1503M02500,0.0901,834,2.50,C,M,2.587,0.04496,6 days
1245,2015-03-19,510050C1503M02550,0.0526,1578,2.55,C,M,2.587,0.04496,6 days
1248,2015-03-19,510050C1503M02600,0.0262,1524,2.60,C,M,2.587,0.04496,6 days
1249,2015-03-19,510050P1503M02600,0.0407,919,2.60,P,M,2.587,0.04496,6 days
...,...,...,...,...,...,...,...,...,...,...
93075,2018-09-21,510050C1809M02400,0.2268,22748,2.40,C,M,2.632,0.02654,5 days
93085,2018-09-21,510050P1809M02500,0.0008,105318,2.50,P,M,2.632,0.02654,5 days
93098,2018-09-21,510050P1809M02450,0.0005,46598,2.45,P,M,2.632,0.02654,5 days
93099,2018-09-21,510050C1809M02450,0.1749,48142,2.45,C,M,2.632,0.02654,5 days


In [59]:
print('到期日小于7天的合约数有{}个。'.format(df2[df2.t.astype('str').apply(lambda x:x[:-5]).astype('int32') < 7].t.shape[0]))
df2 = df2[df2.t.astype('str').apply(lambda x:x[:-5]).astype('int32') >= 7]
df2.loc[:,'t'] = df2.t.astype('str').apply(lambda x:x[:-5]).astype('float64') / 365
#df['t'] = df2.t.astype('str').apply(lambda x:x[:-5]).astype('float64') / 365

到期日小于7天的合约数有6489个。


In [60]:
df2

,date,trade_code,收盘价,成交量,K,call_put,Adjust,close,shibor,t
0,2015-02-09,510050C1509M02200,0.3130,247,2.20,C,M,2.331,0.04335,0.619178
1,2015-02-09,510050P1504M02400,0.1903,395,2.40,P,M,2.331,0.04335,0.197260
2,2015-02-09,510050C1506M02250,0.2366,114,2.25,C,M,2.331,0.04335,0.369863
3,2015-02-09,510050C1506M02300,0.2105,161,2.30,C,M,2.331,0.04335,0.369863
4,2015-02-09,510050C1506M02350,0.1870,106,2.35,C,M,2.331,0.04335,0.369863
...,...,...,...,...,...,...,...,...,...,...
93094,2018-09-21,510050C1812M02650,0.1081,5451,2.65,C,M,2.632,0.02654,0.263014
93095,2018-09-21,510050C1812M02600,0.1341,4462,2.60,C,M,2.632,0.02654,0.263014
93096,2018-09-21,510050C1812M02550,0.1668,4403,2.55,C,M,2.632,0.02654,0.263014
93097,2018-09-21,510050C1812M02500,0.1975,3615,2.50,C,M,2.632,0.02654,0.263014


# 条件约束

In [61]:
pp = df2.copy()

In [62]:
def cons(df): # 计算是否符合条件约束
    # 输出0 代表不符合 
    # 输出1 代表符合
    if df['call_put'] == 'C':
        price = df.close - df.K*exp(-df.shibor*df.t)
        price = max(price,0)
        if price < df['收盘价'] and df['收盘价'] < df.close:
            return 1
        else: return 0 
    else:
        price =  df.K*exp(-df.shibor/100*df.t) - df.close
        price = max(price,0)
        if price < df['收盘价'] and df['收盘价'] < df.K*exp(-df.shibor/100*df.t):
            return 1
        else: return 0 

In [63]:
pp['cons']=pp.apply(cons,axis=1)
pp[pp['cons'] == 0]

,date,trade_code,收盘价,成交量,K,call_put,Adjust,close,shibor,t,cons
451,2015-02-26,510050C1503M02200,0.2530,1246,2.20,C,M,2.450,0.047170,0.073973,0
822,2015-03-09,510050C1503M02200,0.1974,1816,2.20,C,M,2.397,0.047730,0.043836,0
920,2015-03-11,510050C1503M02200,0.1774,1175,2.20,C,M,2.374,0.047730,0.038356,0
977,2015-03-12,510050C1504M02200,0.2501,647,2.20,C,M,2.448,0.047511,0.112329,0
981,2015-03-12,510050C1503M02300,0.1490,848,2.30,C,M,2.448,0.047511,0.035616,0
...,...,...,...,...,...,...,...,...,...,...,...
92953,2018-09-20,510050P1812M02900,0.3535,191,2.90,P,M,2.541,0.026590,0.265753,0
93021,2018-09-21,510050C1810M02250,0.3844,3250,2.25,C,M,2.632,0.026540,0.090411,0
93024,2018-09-21,510050C1810M02400,0.2368,13064,2.40,C,M,2.632,0.026540,0.090411,0
93025,2018-09-21,510050C1810M02350,0.2864,7091,2.35,C,M,2.632,0.026540,0.090411,0


In [64]:
df2 = pp[pp['cons'] == 1]
df2

,date,trade_code,收盘价,成交量,K,call_put,Adjust,close,shibor,t,cons
0,2015-02-09,510050C1509M02200,0.3130,247,2.20,C,M,2.331,0.04335,0.619178,1
1,2015-02-09,510050P1504M02400,0.1903,395,2.40,P,M,2.331,0.04335,0.197260,1
2,2015-02-09,510050C1506M02250,0.2366,114,2.25,C,M,2.331,0.04335,0.369863,1
3,2015-02-09,510050C1506M02300,0.2105,161,2.30,C,M,2.331,0.04335,0.369863,1
4,2015-02-09,510050C1506M02350,0.1870,106,2.35,C,M,2.331,0.04335,0.369863,1
...,...,...,...,...,...,...,...,...,...,...,...
93094,2018-09-21,510050C1812M02650,0.1081,5451,2.65,C,M,2.632,0.02654,0.263014,1
93095,2018-09-21,510050C1812M02600,0.1341,4462,2.60,C,M,2.632,0.02654,0.263014,1
93096,2018-09-21,510050C1812M02550,0.1668,4403,2.55,C,M,2.632,0.02654,0.263014,1
93097,2018-09-21,510050C1812M02500,0.1975,3615,2.50,C,M,2.632,0.02654,0.263014,1


# 数据清洗

加入计算隐含分红率

In [65]:
def iq(df): # 计算隐含分红率
    #q = -log((df.settle+df.exercise_price*exp(-df.interest*df.delta)-df.settle_p)/(df.s0))/df.delta
    q = -log((df.c+df.k*exp(-df.r*df.t)-df.c_p)/(df.s))/df.t
    return q
# df_2['q'] = df_2.apply(iq,axis = 1)

# 只取需要的列
df = df2[['date','trade_code','call_put','t','K','收盘价','close','shibor','Adjust']]
# 给列重新命名
df = df.rename(columns={'date':'trade_date','收盘价':'c','trade_code':'ts_code','shibor':'r','close':'s','K':'k','Adjust':'a'})

# 看涨看跌分类
df_c = df[df['call_put']=='C']
df_p = df[df['call_put']=='P']

# 看跌重命名
df_p = df_p.rename(columns={'c':'c_p','ts_code':'ts_code_p',
                         'call_put':'call_put_p'})

# 外连接
df_p_2 = pd.merge(df_p,df_c,how='outer',on=['trade_date','k','t','r','s','a'])

# 计算隐含波动率
df_p_2['q'] = df_p_2.apply(iq,axis = 1)
                           
# 对没有找到相匹配期权合约的进行空值填充
df_p_2.q.fillna(0,inplace=True)
                           
                           
'''
c_p_together 代表 看跌看涨对应的期权合约 可分为：c_p_1，c_p_2
c_single 代表 无看跌对应的看涨期权合约
p_single 代表 无看涨对应的看跌期权合约
'''
                           
c_p_together = df_p_2.dropna(axis=0)
                           
# 将c_p_together 拆分成两个dataframe
c_p_1 = c_p_together[['trade_date','ts_code','c','call_put','k','t','r','s','q']]
c_p_2 = c_p_together[['trade_date','ts_code_p','c_p','call_put_p','k','t','r','s','q']]
c_p_2 = c_p_2.rename(columns={'c_p':'c','ts_code_p':'ts_code',
                     'call_put_p':'call_put'})

c_single =  df_p_2[df_p_2.ts_code_p.isnull().values==True].dropna(axis=1).drop('a',axis=1)
p_single = df_p_2[df_p_2.ts_code.isnull().values==True].dropna(axis=1).drop('a',axis=1)
                           
# 看跌重命名
p_single = p_single.rename(columns={'c_p':'c','ts_code_p':'ts_code',
                     'call_put_p':'call_put'})
                     
# 全部组合
df_cp = c_p_2.append([c_single,p_single,c_p_1])
df_cp = df_cp.sort_values('trade_date').reset_index(drop=True)

In [66]:
df_cp

,trade_date,ts_code,c,call_put,k,t,r,s,q
0,2015-02-09,510050P1504M02400,0.1903,P,2.40,0.197260,0.04335,2.331,0.089817
1,2015-02-09,510050C1506M02300,0.2105,C,2.30,0.369863,0.04335,2.331,0.053040
2,2015-02-09,510050C1506M02350,0.1870,C,2.35,0.369863,0.04335,2.331,0.043117
3,2015-02-09,510050C1506M02400,0.1652,C,2.40,0.369863,0.04335,2.331,0.043111
4,2015-02-09,510050C1506M02250,0.2366,C,2.25,0.369863,0.04335,2.331,0.050681
...,...,...,...,...,...,...,...,...,...
77814,2018-09-21,510050P1810M02650,0.0700,P,2.65,0.090411,0.02654,2.632,0.017458
77815,2018-09-21,510050P1810M02600,0.0450,P,2.60,0.090411,0.02654,2.632,0.014008
77816,2018-09-21,510050P1810M02550,0.0282,P,2.55,0.090411,0.02654,2.632,0.015187
77817,2018-09-21,510050P1810M02450,0.0102,P,2.45,0.090411,0.02654,2.632,0.017545


In [68]:
#弃用
# https://blog.csdn.net/u012572955/article/details/53380263
# 将每一天日期对应日期找出来   
# import time
# import datetime
# def getBetweenDay(begin_date,end_date):
#     date_list = []
#     begin_date = datetime.datetime.strptime(begin_date, "%Y-%m-%d")
#     # time.strftime('%Y-%m-%d',time.localtime(time.time()))
#     end_date = datetime.datetime.strptime(end_date, "%Y-%m-%d")
#     while begin_date <= end_date:
#         if str(begin_date.weekday()) not in '56': # 5,6代表周六周日
#             date_str = begin_date.strftime("%Y-%m-%d")
#             date_list.append(date_str)
#         begin_date += datetime.timedelta(days=1)
#     return date_list
# continue_time = getBetweenDay("2015-02-09","2018-09-21")

In [70]:
df_final = df_cp.copy()

In [71]:
df_final

,trade_date,ts_code,c,call_put,k,t,r,s,q
0,2015-02-09,510050P1504M02400,0.1903,P,2.40,0.197260,0.04335,2.331,0.089817
1,2015-02-09,510050C1506M02300,0.2105,C,2.30,0.369863,0.04335,2.331,0.053040
2,2015-02-09,510050C1506M02350,0.1870,C,2.35,0.369863,0.04335,2.331,0.043117
3,2015-02-09,510050C1506M02400,0.1652,C,2.40,0.369863,0.04335,2.331,0.043111
4,2015-02-09,510050C1506M02250,0.2366,C,2.25,0.369863,0.04335,2.331,0.050681
...,...,...,...,...,...,...,...,...,...
77814,2018-09-21,510050P1810M02650,0.0700,P,2.65,0.090411,0.02654,2.632,0.017458
77815,2018-09-21,510050P1810M02600,0.0450,P,2.60,0.090411,0.02654,2.632,0.014008
77816,2018-09-21,510050P1810M02550,0.0282,P,2.55,0.090411,0.02654,2.632,0.015187
77817,2018-09-21,510050P1810M02450,0.0102,P,2.45,0.090411,0.02654,2.632,0.017545


In [72]:
#根据BS公式计算期权价值
def bsm_value(s,k,t,r,sigma,q,option_type):
    d1 = ( log( s/k ) + ( r -q + 0.5*sigma**2 )*t )/( sigma*sqrt(t) )
    d2 = ( log( s/k ) + ( r -q - 0.5*sigma**2 )*t )/( sigma*sqrt(t) )
    if option_type.lower() == 'c':
        value = (s*exp(-q*t)*stats.norm.cdf( d1) - k*exp( -r*t )*stats.norm.cdf( d2))
    else:
        value = k * exp(-r * t) * stats.norm.cdf(-d2) - s*exp(-q*t) * stats.norm.cdf(-d1)
    return value

#二分法求隐含波动率
def bsm_imp_vol(s,k,t,r,c,q,option_type):
    # print(c)
    c_est = 0 # 期权价格估计值
    top = 1  #波动率上限
    floor = 0  #波动率下限
    sigma = ( floor + top )/2 #波动率初始值
    count = 0 # 计数器
    while abs( c - c_est ) > 0.0001:
        c_est = bsm_value(s,k,t,r,sigma,q,option_type) 
        #根据价格判断波动率是被低估还是高估，并对波动率做修正
        count += 1           
        if count > 100: # 时间价值为0的期权是算不出隐含波动率的，因此迭代到一定次数就不再迭代了
            sigma = 0
            break
        if c - c_est > 0: #f(x)>0
            floor = sigma
            sigma = ( sigma + top )/2
        else:
            top = sigma
            sigma = ( sigma + floor )/2
    return sigma  

def cal_iv(df): # 计算主程序
    option_list = df.ts_code.tolist()
    # print(option_list)
    df = df.set_index('ts_code')
    alist = []

    for option in option_list:
        s = df.loc[option,'s']
        k = df.loc[option,'k']
        t = df.loc[option,'t']
        r = df.loc[option,'r']
        c = df.loc[option,'c']
        q = df.loc[option,'q']
        option_type = df.loc[option,'call_put']
        sigma = bsm_imp_vol(s,k,t,r,c,q,option_type)
        alist.append(sigma)
    df['iv'] = alist
    return df  

In [75]:
def cal_v(df,long_time):
    df_final = pd.DataFrame()
    for ti in long_time:
        d_temp = df[df.trade_date==ti]
        d_temp = cal_iv(d_temp) # 计算隐含波动率
        # print(d_temp)
        # 处理日期
        d_temp['ts_code']=d_temp.index
        d_temp = d_temp.reset_index(drop=True)
        df_final = df_final.append(d_temp)
        df_final = df_final.reset_index(drop=True)
    return df_final

In [69]:
# 根据交易日进行分组
long_time = df_cp.groupby('trade_date').count().index.astype('str').tolist()
# 遍历所有交易，计算iv
df_final_iv = cal_v(df_final,long_time)

In [77]:
df_final_iv

,trade_date,c,call_put,k,t,r,s,q,iv,ts_code
0,2015-02-09,0.1903,P,2.40,0.197260,0.04335,2.331,0.089817,0.341553,510050P1504M02400
1,2015-02-09,0.2105,C,2.30,0.369863,0.04335,2.331,0.053040,0.361694,510050C1506M02300
2,2015-02-09,0.1870,C,2.35,0.369863,0.04335,2.331,0.043117,0.355469,510050C1506M02350
3,2015-02-09,0.1652,C,2.40,0.369863,0.04335,2.331,0.043111,0.349731,510050C1506M02400
4,2015-02-09,0.2366,C,2.25,0.369863,0.04335,2.331,0.050681,0.362549,510050C1506M02250
...,...,...,...,...,...,...,...,...,...,...
77814,2018-09-21,0.0700,P,2.65,0.090411,0.02654,2.632,0.017458,0.199219,510050P1810M02650
77815,2018-09-21,0.0450,P,2.60,0.090411,0.02654,2.632,0.014008,0.194824,510050P1810M02600
77816,2018-09-21,0.0282,P,2.55,0.090411,0.02654,2.632,0.015187,0.197754,510050P1810M02550
77817,2018-09-21,0.0102,P,2.45,0.090411,0.02654,2.632,0.017545,0.214844,510050P1810M02450


# 把成交量也写上

In [78]:
df3 = df2.rename(columns={'date':'trade_date','trade_code':'ts_code','成交量':'volume'})
df3 = df3[['trade_date','ts_code','volume']].reset_index(drop=True)

In [79]:
df_pre = pd.merge(df_final_iv,df3,on=['trade_date','ts_code'],how='left')

In [80]:
df_pre

,trade_date,c,call_put,k,t,r,s,q,iv,ts_code,volume
0,2015-02-09,0.1903,P,2.40,0.197260,0.04335,2.331,0.089817,0.341553,510050P1504M02400,395
1,2015-02-09,0.2105,C,2.30,0.369863,0.04335,2.331,0.053040,0.361694,510050C1506M02300,161
2,2015-02-09,0.1870,C,2.35,0.369863,0.04335,2.331,0.043117,0.355469,510050C1506M02350,106
3,2015-02-09,0.1652,C,2.40,0.369863,0.04335,2.331,0.043111,0.349731,510050C1506M02400,225
4,2015-02-09,0.2366,C,2.25,0.369863,0.04335,2.331,0.050681,0.362549,510050C1506M02250,114
...,...,...,...,...,...,...,...,...,...,...,...
77814,2018-09-21,0.0700,P,2.65,0.090411,0.02654,2.632,0.017458,0.199219,510050P1810M02650,23415
77815,2018-09-21,0.0450,P,2.60,0.090411,0.02654,2.632,0.014008,0.194824,510050P1810M02600,66847
77816,2018-09-21,0.0282,P,2.55,0.090411,0.02654,2.632,0.015187,0.197754,510050P1810M02550,83216
77817,2018-09-21,0.0102,P,2.45,0.090411,0.02654,2.632,0.017545,0.214844,510050P1810M02450,69383


In [81]:
# 重新排一下序
t_columns = ['trade_date', 'ts_code', 'call_put','volume','t', 'k', 'c', 's', 'r', 'q','iv']
# df_final.reindex(columns=t_column)
df_pre = df_pre.reindex(columns=t_columns)

In [82]:
df_pre

,trade_date,ts_code,call_put,volume,t,k,c,s,r,q,iv
0,2015-02-09,510050P1504M02400,P,395,0.197260,2.40,0.1903,2.331,0.04335,0.089817,0.341553
1,2015-02-09,510050C1506M02300,C,161,0.369863,2.30,0.2105,2.331,0.04335,0.053040,0.361694
2,2015-02-09,510050C1506M02350,C,106,0.369863,2.35,0.1870,2.331,0.04335,0.043117,0.355469
3,2015-02-09,510050C1506M02400,C,225,0.369863,2.40,0.1652,2.331,0.04335,0.043111,0.349731
4,2015-02-09,510050C1506M02250,C,114,0.369863,2.25,0.2366,2.331,0.04335,0.050681,0.362549
...,...,...,...,...,...,...,...,...,...,...,...
77814,2018-09-21,510050P1810M02650,P,23415,0.090411,2.65,0.0700,2.632,0.02654,0.017458,0.199219
77815,2018-09-21,510050P1810M02600,P,66847,0.090411,2.60,0.0450,2.632,0.02654,0.014008,0.194824
77816,2018-09-21,510050P1810M02550,P,83216,0.090411,2.55,0.0282,2.632,0.02654,0.015187,0.197754
77817,2018-09-21,510050P1810M02450,P,69383,0.090411,2.45,0.0102,2.632,0.02654,0.017545,0.214844


# 描述统计模块
不影响画图功能

## Mi 在值程度

In [83]:
def m_cal(df): # 在值公式
    m = log(df.k/df.s*exp(df.r*df.t))
    return m  

In [84]:
df_m = df_pre.copy()
df_m['M'] = df_m.apply(m_cal, axis=1) # 计算在值程度
c_put = df_m[df_m['call_put']=='C']
p_cal = df_m[df_m['call_put']=='P']
df_m.to_csv('iv_m_data_final.csv')

In [85]:
df_m

,trade_date,ts_code,call_put,volume,t,k,c,s,r,q,iv,M
0,2015-02-09,510050P1504M02400,P,395,0.197260,2.40,0.1903,2.331,0.04335,0.089817,0.341553,0.037723
1,2015-02-09,510050C1506M02300,C,161,0.369863,2.30,0.2105,2.331,0.04335,0.053040,0.361694,0.002645
2,2015-02-09,510050C1506M02350,C,106,0.369863,2.35,0.1870,2.331,0.04335,0.043117,0.355469,0.024152
3,2015-02-09,510050C1506M02400,C,225,0.369863,2.40,0.1652,2.331,0.04335,0.043111,0.349731,0.045205
4,2015-02-09,510050C1506M02250,C,114,0.369863,2.25,0.2366,2.331,0.04335,0.050681,0.362549,-0.019334
...,...,...,...,...,...,...,...,...,...,...,...,...
77814,2018-09-21,510050P1810M02650,P,23415,0.090411,2.65,0.0700,2.632,0.02654,0.017458,0.199219,0.009215
77815,2018-09-21,510050P1810M02600,P,66847,0.090411,2.60,0.0450,2.632,0.02654,0.014008,0.194824,-0.009833
77816,2018-09-21,510050P1810M02550,P,83216,0.090411,2.55,0.0282,2.632,0.02654,0.015187,0.197754,-0.029251
77817,2018-09-21,510050P1810M02450,P,69383,0.090411,2.45,0.0102,2.632,0.02654,0.017545,0.214844,-0.069256


看跌期权在值程度与抗张期权相反

In [95]:
p_call = p_cal.copy()
p_call.loc[:,'M'] = p_cal.M.apply(lambda x: -x)

未完工部分

In [100]:
# 重复代码
# 样本统计
#def sample_count(df):
#    return [df.shape[0],round(df.iv.mean(),4)]

# 样本量百分比
def sample_percent(df,type_s):
    return round(df[type_s].volume.sum() / c_put.volume.sum() * 100,2) 

In [101]:
# 成交量百分比
def volume_percent(df):
    deep_s = df_m.M <= -0.12
    normal_s = (df_m.M > -0.12) & (df_m.M <= -0.03)
    equal_s = (df_m.M > -0.03) & (df_m.M < 0.03)
    virtual_s = (df_m.M >= 0.03) & (df_m.M < 0.12)
    high_s = df_m.M > 0.12
    return [round(df[i].volume.sum() / df_m.volume.sum(),4)
            for i in [deep_s,normal_s,equal_s,virtual_s,high_s]]

volume_p = volume_percent(df_m)
volume_p # 深度虚值，虚值，平值，实值，深度实值

[0.0131, 0.1677, 0.5825, 0.2117, 0.025]

In [102]:
# 样本统计 
def sample_count(df):
    return [df.shape[0],round(df.iv.mean(),4)]

def descriptive(df,type_s):
    if type_s == 'C':
        temp = df[df['call_put']=='C']
        # print(temp.shape)
        # print(df.shape)
    else:
        temp = df[df['call_put']=='P'].copy()
        temp.loc[:,'M'] = df.M.apply(lambda x: -x) # 看跌期权在值与看涨相反
        # print(temp.shape)
    # def sample_percent(df,type_s):
    #     return round(df[type_s].volume.sum() / c_put.volume.sum() * 100,2)
    # 长短期划分
    short = ((temp.t*365) < 60)
    medium = (temp.t*365 >= 60) & (temp.t*365 <= 180)
    long = ((temp.t*365) > 180)
    
    # 设定上下限
    top = 0.12 
    mid = 0.03
    
    # 深度实值
    deep_s = temp.M<=-top
    s_1 = temp[deep_s & short]
    m_1 = temp[deep_s & medium]
    l_1 = temp[deep_s & long]
    deep_1ist = [sample_count(i) for i in [s_1,m_1,l_1]]

        # normal_s = (c_put.M > -0.12) & (c_put.M <= -0.03)
    # 实值
    # print(temp.shape)
    normal_s = ((temp.M>-top)&(temp.M<=-mid))

    s_2 = temp[normal_s & short]
    # print(s_2.shape)
    m_2 = temp[normal_s & medium]
    l_2 = temp[normal_s & long]
    normal_1ist = [sample_count(i) for i in [s_2,m_2,l_2]]
    # print(s_2)
    
    # 平值
    equal_s = (temp.M>-mid)&(temp.M<mid)
    s_3 = temp[equal_s & short]
    m_3 = temp[equal_s & medium]
    l_3 = temp[equal_s & long]
    equal_1ist = [sample_count(i) for i in [s_3,m_3,l_3]]
    
    # 虚值
    virtual_s = (temp.M>=mid)&(temp.M<top)
    s_4 = temp[virtual_s & short]
    m_4 = temp[virtual_s & medium]
    l_4 = temp[virtual_s & long]
    virtual_1ist = [sample_count(i) for i in [s_4,m_4,l_4]]
    
    
    #深度虚值
    high_s = temp.M>=top
    s_5 = temp[high_s & short]
    m_5 = temp[high_s & medium]
    l_5 = temp[high_s & long]
    high_1ist = [sample_count(i) for i in [s_5,m_5,l_5]]
    
    num = [deep_1ist,normal_1ist,equal_1ist, virtual_1ist, high_1ist]
    return num

In [103]:
des_call = descriptive(df_m,'C')
des_put = descriptive(df_m,'P')

## 描述性统计 des_call 看涨期权， des_put 看跌期权  
第一行 深度虚值 长中短期 样本量 隐含波动率  
第二行 虚值 长中短期 样本量 隐含波动率 …………

In [105]:
des_call, des_put

([[[1195, 0.3806], [1047, 0.2994], [60, 0.3524]],
  [[3749, 0.2638], [3494, 0.2267], [1078, 0.2208]],
  [[4807, 0.2192], [3907, 0.2143], [1733, 0.2056]],
  [[4900, 0.2591], [4561, 0.236], [1932, 0.2211]],
  [[3114, 0.432], [3352, 0.3372], [519, 0.3058]]],
 [[[1932, 0.47], [1506, 0.3628], [292, 0.3335]],
  [[3517, 0.2762], [3588, 0.2325], [1734, 0.2126]],
  [[4670, 0.2176], [3855, 0.211], [1748, 0.2054]],
  [[5235, 0.2554], [4295, 0.2234], [1303, 0.2288]],
  [[2290, 0.3778], [2268, 0.3074], [138, 0.3745]]])

In [109]:
sample_volume = df_m.shape[0]
trade_volume = df_m.volume.sum()
print("样本总量为：",sample_volume)
print("成交总量为：",trade_volume)

样本总量为： 77819
成交总量为： 409558722


In [110]:
sample_count= []
for i in zip(des_call,des_put):
    
    sample_count.append(i[0][0][0]+i[0][1][0]+i[0][2][0]+i[1][0][0]+i[1][1][0]+i[1][2][0])
sample_percent = [round(i / sample_volume,4) for i in sample_count]

In [111]:
sample_percent

[0.0775, 0.2205, 0.2663, 0.2856, 0.1501]

In [112]:
volume_p,sample_percent

([0.0131, 0.1677, 0.5825, 0.2117, 0.025],
 [0.0775, 0.2205, 0.2663, 0.2856, 0.1501])

In [113]:
sample_percent

[0.0775, 0.2205, 0.2663, 0.2856, 0.1501]

# 最终形态

In [130]:
df_t = df_pre.copy()

,trade_date,ts_code,call_put,volume,t,k,c,s,r,q,iv
0,2015-02-09,510050P1504M02400,P,395,0.197260,2.40,0.1903,2.331,0.04335,0.089817,0.341553
1,2015-02-09,510050C1506M02300,C,161,0.369863,2.30,0.2105,2.331,0.04335,0.053040,0.361694
2,2015-02-09,510050C1506M02350,C,106,0.369863,2.35,0.1870,2.331,0.04335,0.043117,0.355469
3,2015-02-09,510050C1506M02400,C,225,0.369863,2.40,0.1652,2.331,0.04335,0.043111,0.349731
4,2015-02-09,510050C1506M02250,C,114,0.369863,2.25,0.2366,2.331,0.04335,0.050681,0.362549
...,...,...,...,...,...,...,...,...,...,...,...
77814,2018-09-21,510050P1810M02650,P,23415,0.090411,2.65,0.0700,2.632,0.02654,0.017458,0.199219
77815,2018-09-21,510050P1810M02600,P,66847,0.090411,2.60,0.0450,2.632,0.02654,0.014008,0.194824
77816,2018-09-21,510050P1810M02550,P,83216,0.090411,2.55,0.0282,2.632,0.02654,0.015187,0.197754
77817,2018-09-21,510050P1810M02450,P,69383,0.090411,2.45,0.0102,2.632,0.02654,0.017545,0.214844


# 画图

In [121]:
def data_pivot(df): # 数据透视
    df = df.reset_index()
    option_type = 'C' # 具有相同执行价格、相同剩余到期时间的看涨看跌期权隐含波动率相等，因此算一个就够了
    df = df[df['call_put']==option_type]
    df = df.drop(['ts_code','trade_date','c','s','r','call_put','q'],axis=1)
    df['t'] = df['t']*365
    df['t'] = df['t'].astype(int)
    df = df.pivot_table(index=["k"],columns=["t"],values=["iv"])
    df.columns = df.columns.droplevel(0)
    df.index.name = None
    df = df.reset_index()
    df = df.rename(columns={'index':'k'})

    return df

def fitting(df): # 多项式拟合    
    col_list = df.columns
    for i in range(df.shape[1]-1):
        x_col = col_list[0]
        y_col = col_list[i+1]
        df1 = df.dropna(subset=[y_col])
        
        x = df1.iloc[:,0]
        y = df1.iloc[:,i+1]

        degree = 2
                
        weights = np.polyfit(x, y, degree)
        model = np.poly1d(weights)
        predict = np.poly1d(model)
        x_given_list = df[pd.isnull(df[y_col]) == True][x_col].tolist()
        # 所有空值对应的k组成列表
        for x_given in x_given_list:
            y_predict = predict(x_given)
            df.loc[df[x_col]==x_given, y_col] = y_predict
    return df

In [122]:
def im_surface(df): # 波动率曲面作图
    # df = plot_df()
    df = fitting(df)    
    #df.to_excel('iv_fitting.xlsx')
    df = df.set_index('k')

    y = np.array(df.index)
    x = np.array(df.columns)
    fig = go.Figure(data=[go.Surface(z=df.values, x=x, y=y)])

    fig.update_layout(scene = dict(
                    xaxis_title='剩余期限',
                    yaxis_title='执行价格',
                    zaxis_title='隐含波动率'),
                    width=1400,
                    margin=dict(r=20, b=10, l=10, t=10),
                    )
    # fig.write_image("fig1.jpg")
    plotly.offline.plot(fig)

def smile_plot(df): # 波动率微笑作图
    # df = plot_df()
    df = df.set_index('k')
    df = df.stack().reset_index()
    df.columns = ['k', 'days', 'iv']
    fig = px.line(df, x="k", y="iv", color="days",line_shape="spline")
    plotly.offline.plot(fig)

In [ ]:
date_info = '2018-07-23' # 只需要指定日期 2015-02-09 至 2018-09-21 任意一天即可画隐含波动率曲面图
df = df_t[df_t.trade_date == date_info] # 获取具体某一天的所有期权合约
df['ts_code']=df.index
df = df.reset_index(drop=True)
df = data_pivot(df) # 数据透视表
df = fitting(df)
im_surface(df)